# API Documentation Generation

> Generate module overviews with formatted signatures for nbdev projects

In [ ]:
#| default_exp api_docs

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from __future__ import annotations
from pathlib import Path
from typing import List, Dict, Any, Optional
from nbdev.config import get_config
from cjm_nbdev_overview.core import *
from cjm_nbdev_overview.parsers import *
from cjm_nbdev_overview.tree import *
from cjm_nbdev_overview.dependencies import *
from dataclasses import dataclass
import textwrap

from execnb.nbio import read_nb, write_nb, mk_cell, new_nb
from fastcore.xtras import Path
import re

## Formatting Functions

In [ ]:
#| export
def format_function_doc(func: FunctionInfo,             # Function information
                       indent: str = ""                 # Indentation prefix
                       ) -> str:                        # Formatted documentation
    "Format a function with its signature for documentation"
    lines = []
    
    # Add the code block with signature
    lines.append("```python")
    
    # Add decorators if any
    for decorator in func.decorators:
        lines.append(f"{indent}@{decorator}")
    
    # Add the signature
    sig_lines = func.signature.split('\n')
    for line in sig_lines:
        lines.append(f"{indent}{line}")
    
    # Add docstring if present
    if func.docstring:
        docstring_lines = func.docstring.strip().split('\n')
        if len(docstring_lines) == 1:
            lines.append(f'{indent}    "{docstring_lines[0]}"')
        else:
            lines.append(f'{indent}    """')
            for doc_line in docstring_lines:
                lines.append(f'{indent}    {doc_line}')
            lines.append(f'{indent}    """')
    
    lines.append("```")
    
    return '\n'.join(lines)

In [ ]:
#| export
def format_class_doc(cls: ClassInfo                     # Class information
                    ) -> str:                           # Formatted documentation
    "Format a class with its signature and methods for documentation"
    lines = []
    
    lines.append("```python")
    
    # Add decorators if any
    for decorator in cls.decorators:
        lines.append(f"@{decorator}")
    
    # Add class signature
    lines.append(cls.signature)
    
    # Add class docstring if present
    if cls.docstring:
        docstring_lines = cls.docstring.strip().split('\n')
        if len(docstring_lines) == 1:
            lines.append(f'    "{docstring_lines[0]}"')
        else:
            lines.append('    """')
            for doc_line in docstring_lines:
                lines.append(f'    {doc_line}')
            lines.append('    """')
    
    # Add dataclass attributes if this is a dataclass
    if 'dataclass' in cls.decorators and cls.attributes:
        lines.append("    ")
        for attr in cls.attributes:
            attr_line = f"    {attr.name}"
            if attr.type_hint:
                attr_line += f": {attr.type_hint}"
            if attr.value and attr.value != "None":
                attr_line += f" = {attr.value}"
            if attr.comment:
                attr_line += f"  # {attr.comment}"
            lines.append(attr_line)
    
    # Add all methods
    if cls.methods:
        lines.append("    ")
        # Show __init__ and public methods only
        shown_methods = [m for m in cls.methods if not m.name.startswith('_') or m.name == '__init__']
        
        for i, method in enumerate(shown_methods):  # Show all methods, no truncation
            if i > 0:
                lines.append("    ")
            
            # Format method signature
            method_lines = method.signature.split('\n')
            for j, line in enumerate(method_lines):
                # Remove the 'def' from the first line if it's there
                if j == 0 and line.strip().startswith('def'):
                    lines.append(f"    {line.strip()}")
                else:
                    lines.append(f"    {line}")
            
            # Add method docstring
            if method.docstring:
                lines.append(f'        "{method.docstring.strip()}"')
    
    lines.append("```")
    
    return '\n'.join(lines)

In [ ]:
#| export  
def format_variable_doc(var: VariableInfo               # Variable information
                       ) -> str:                        # Formatted documentation
    "Format a variable for documentation"
    parts = [var.name]
    
    if var.type_hint:
        parts.append(f": {var.type_hint}")
    
    if var.value:
        parts.append(f" = {var.value}")
    
    if var.comment:
        parts.append(f"  # {var.comment}")
    
    return ''.join(parts)

## Module Documentation Generation

In [ ]:
#| export
def _generate_module_header(module: ModuleInfo          # Module information
                          ) -> List[str]:               # Header lines
    "Generate module title and description lines"
    lines = []
    
    # Add module title
    if module.title:
        lines.append(f"### {module.title} (`{module.path.name}`)")
    else:
        lines.append(f"### {module.name} (`{module.path.name}`)")
    
    # Add description
    if module.description:
        lines.append(f"> {module.description}")
    
    return lines

In [ ]:
#| export
def _generate_import_statement(module: ModuleInfo       # Module information
                             ) -> List[str]:            # Import statement lines
    "Generate import statement lines for a module"
    lines = []
    lines.append("\n#### Import\n")
    
    try:
        from nbdev.config import get_config
        cfg = get_config()
        
        # For nested modules, we need to construct the proper path
        # If module path is nbs/actions/button.ipynb, the Python module should be at lib/actions/button.py
        relative_path = module.path.relative_to(cfg.nbs_path).with_suffix('.py')
        python_module_path = cfg.lib_path / relative_path
        
        if python_module_path.exists():
            # Read the Python file to extract __all__
            with open(python_module_path, 'r', encoding='utf-8') as f:
                content = f.read()
            
            # Find the __all__ definition
            import ast
            try:
                tree = ast.parse(content)
                all_items = []
                
                for node in ast.walk(tree):
                    if isinstance(node, ast.Assign):
                        for target in node.targets:
                            if isinstance(target, ast.Name) and target.id == '__all__':
                                if isinstance(node.value, ast.List):
                                    all_items = [
                                        elt.s if isinstance(elt, ast.Str) else elt.value
                                        for elt in node.value.elts
                                        if isinstance(elt, (ast.Str, ast.Constant))
                                    ]
                                break
                
                if all_items:
                    # Create import statement - replace hyphens with underscores for valid Python imports
                    package_name = cfg.lib_name.replace('-', '_')
                    # Convert file path to module path (e.g., actions/button.py -> actions.button)
                    module_path_parts = relative_path.with_suffix('').parts
                    full_module_path = f"{package_name}.{'.'.join(module_path_parts)}"
                    
                    lines.append("```python")
                    lines.append(f"from {full_module_path} import (")
                    
                    # Add each item on a separate line with proper formatting
                    for i, item in enumerate(all_items):
                        if i == len(all_items) - 1:
                            lines.append(f"    {item}")
                        else:
                            lines.append(f"    {item},")
                    
                    lines.append(")")
                    lines.append("```")
                else:
                    # Fallback if __all__ not found - replace hyphens with underscores
                    package_name = cfg.lib_name.replace('-', '_')
                    module_path_parts = relative_path.with_suffix('').parts
                    full_module_path = f"{package_name}.{'.'.join(module_path_parts)}"
                    lines.append("```python")
                    lines.append(f"from {full_module_path} import *")
                    lines.append("```")
            
            except Exception:
                # Fallback if parsing fails - replace hyphens with underscores
                package_name = cfg.lib_name.replace('-', '_')
                module_path_parts = relative_path.with_suffix('').parts
                full_module_path = f"{package_name}.{'.'.join(module_path_parts)}"
                lines.append("```python")
                lines.append(f"from {full_module_path} import *")
                lines.append("```")
        else:
            # Fallback if Python file doesn't exist
            module_path_parts = relative_path.with_suffix('').parts
            module_name = '.'.join(module_path_parts)
            lines.append("```python")
            lines.append(f"# No corresponding Python module found for {module_name}")
            lines.append("```")
    except Exception:
        # Final fallback
        lines.append("```python")
        lines.append(f"# Import statements not available")
        lines.append("```")
    
    return lines

In [ ]:
#| export
def _filter_module_items(module: ModuleInfo,            # Module information
                        show_all: bool = False          # Show all items including private
                        ) -> tuple:                     # (functions, classes, variables)
    "Filter module items based on show_all and is_exported flags"
    if show_all:
        functions = module.functions
        classes = module.classes  
        variables = module.variables
    else:
        functions = [f for f in module.functions if f.is_exported]
        classes = [c for c in module.classes if c.is_exported]
        variables = [v for v in module.variables if v.is_exported]
    
    return functions, classes, variables

In [ ]:
#| export
def _generate_functions_section(functions: List[FunctionInfo]   # List of functions
                              ) -> List[str]:                   # Section lines
    "Generate the functions section of module documentation"
    lines = []
    if functions:
        lines.append("\n#### Functions\n")
        for func in functions:
            lines.append(format_function_doc(func))
            lines.append("")
    return lines

In [ ]:
#| export
def _generate_classes_section(classes: List[ClassInfo]          # List of classes
                            ) -> List[str]:                     # Section lines
    "Generate the classes section of module documentation"
    lines = []
    if classes:
        lines.append("#### Classes\n")
        for cls in classes:
            lines.append(format_class_doc(cls))
            lines.append("")
    return lines

In [ ]:
#| export
def _generate_variables_section(variables: List[VariableInfo]   # List of variables
                              ) -> List[str]:                   # Section lines
    "Generate the variables section of module documentation"
    lines = []
    if variables:
        lines.append("#### Variables\n")
        lines.append("```python")
        for var in variables:
            lines.append(format_variable_doc(var))
        lines.append("```")
    return lines

In [ ]:
#| export
def generate_module_overview(module: ModuleInfo,        # Module information
                           show_all: bool = False       # Show all items including private
                           ) -> str:                    # Module overview markdown
    "Generate a markdown overview for a module"
    lines = []
    
    # Generate header (title and description)
    lines.extend(_generate_module_header(module))
    
    # Generate import statement
    lines.extend(_generate_import_statement(module))
    
    # Filter items based on show_all and is_exported
    functions, classes, variables = _filter_module_items(module, show_all)
    
    # Generate sections for functions, classes, and variables
    lines.extend(_generate_functions_section(functions))
    lines.extend(_generate_classes_section(classes))
    lines.extend(_generate_variables_section(variables))
    
    return '\n'.join(lines)

In [ ]:
#| export
def generate_project_api_docs(path: Path = None,        # Project path (defaults to nbs_path)
                            show_all: bool = False      # Show all items including private
                            ) -> str:                   # Full API documentation
    "Generate API documentation for all modules in a project"
    if path is None:
        cfg = get_config()
        path = cfg.nbs_path
    
    lines = []
    lines.append("## API Reference\n")
    
    # Get all notebooks
    notebooks = get_notebook_files(path, recursive=True)
    
    # Parse and document each notebook
    for nb_path in notebooks:
        # Skip index notebooks
        if nb_path.stem in ['index']:
            continue
            
        try:
            module_info = parse_notebook(nb_path)
            
            # Only include if it has exported content
            has_exports = any([
                any(f.is_exported for f in module_info.functions),
                any(c.is_exported for c in module_info.classes),
                any(v.is_exported for v in module_info.variables)
            ])
            
            if has_exports or show_all:
                lines.append(generate_module_overview(module_info, show_all))
                lines.append("\n---\n")
        except Exception as e:
            print(f"Error parsing {nb_path}: {e}")
            continue
    
    return '\n'.join(lines)

## Index Update Function

The `update_index_module_docs` function can automatically update your project's index.ipynb with module documentation:

In [ ]:
#| export
def _filter_cells_removing_sections(cells: List,               # List of notebook cells
                                   start_marker: str            # Section marker to remove
                                   ) -> List:                   # Filtered cells
    "Remove all cells from a section marked by start_marker until the next ## section"
    cells_to_keep = []
    skip_until_next_section = False
    
    for cell in cells:
        if cell.cell_type == 'markdown':
            source = cell.source.strip()
            # Check if this is the start of a section to remove
            if source.startswith(start_marker):
                skip_until_next_section = True
                continue
            # Check if we've reached a new top-level section (## but not ### or more)
            elif skip_until_next_section and re.match(r'^##\s+(?!#)', source) and not source.startswith(start_marker):
                skip_until_next_section = False
        
        # Keep the cell if we're not in a section to remove
        if not skip_until_next_section:
            cells_to_keep.append(cell)
    
    return cells_to_keep

In [ ]:
#| export
def _sort_notebooks_by_prefix(notebooks: List[Path]             # List of notebook paths
                             ) -> List[Path]:                   # Sorted notebook paths
    "Sort notebooks by their numeric prefix, putting non-numbered notebooks at the end"
    def sort_key(
        nb_path: Path  # TODO: Add description
    ) -> tuple:  # TODO: Add return description
        "TODO: Add function description"
        match = re.match(r'^(\d+)', nb_path.stem)
        if match:
            return (int(match.group(1)), nb_path.stem)
        return (999, nb_path.stem)  # Put non-numbered notebooks at the end
    
    return sorted(notebooks, key=sort_key)

In [ ]:
#| export
def _get_notebooks_with_exports(notebooks: List[Path]          # List of notebook paths
                               ) -> List[Path]:                 # Notebooks with exported content
    "Filter notebooks to only include those with exported content"
    notebooks_with_exports = []
    
    for nb_path in notebooks:
        # Skip index notebooks
        if nb_path.stem in ['index', '00_index']:
            continue
            
        try:
            module_info = parse_notebook(nb_path)
            
            # Only include if it has exported content
            has_exports = any([
                any(f.is_exported for f in module_info.functions),
                any(c.is_exported for c in module_info.classes),
                any(v.is_exported for v in module_info.variables)
            ])
            
            if has_exports:
                notebooks_with_exports.append(nb_path)
        except Exception as e:
            print(f"Error parsing {nb_path}: {e}")
            continue
    
    return notebooks_with_exports

In [ ]:
#| export
def _generate_module_overview_cells(notebooks: List[Path]      # List of notebook paths
                                   ) -> List:                   # List of notebook cells
    "Generate markdown cells containing module overview documentation"
    module_cells = []
    
    # Create the module overview header cell
    header_cell = mk_cell("## Module Overview\n\nDetailed documentation for each module in the project:", 
                         cell_type='markdown')
    module_cells.append(header_cell)
    
    # Generate overview for each module
    for nb_path in notebooks:
        try:
            module_info = parse_notebook(nb_path)
            overview_md = generate_module_overview(module_info)
            overview_cell = mk_cell(overview_md, cell_type='markdown')
            module_cells.append(overview_cell)
        except Exception as e:
            print(f"Error parsing {nb_path}: {e}")
            continue
    
    return module_cells

In [ ]:
#| export
def update_index_module_docs(index_path: Path = None,          # Path to index.ipynb (defaults to nbs/index.ipynb)
                           start_marker: str = "## Module Overview"  # Marker to identify module docs section
                           ) -> None:                          # Updates index.ipynb in place
    "Update the module documentation section in index.ipynb"
    if index_path is None:
        cfg = get_config()
        index_path = cfg.nbs_path / "index.ipynb"
    
    # Read the existing notebook
    nb = read_nb(index_path)
    
    # Filter out existing module overview sections
    cells_to_keep = _filter_cells_removing_sections(nb.cells, start_marker)
    
    # Get all notebooks and process them
    notebooks = get_notebook_files(index_path.parent, recursive=True)
    sorted_notebooks = _sort_notebooks_by_prefix(notebooks)
    notebooks_with_exports = _get_notebooks_with_exports(sorted_notebooks)
    
    # Generate new module overview cells
    module_cells = _generate_module_overview_cells(notebooks_with_exports)
    
    # Rebuild the notebook with the new module overview at the end
    nb.cells = cells_to_keep + module_cells
    
    # Write the updated notebook
    write_nb(nb, index_path)

In [ ]:
# Example: Update the index.ipynb with module documentation
# update_index_module_docs()  # This will update nbs/index.ipynb

# Or specify a custom path:
# update_index_module_docs(Path("../other_project/nbs/index.ipynb"))

## Comprehensive Index Update Functions

Additional functions to update index.ipynb with project structure, dependencies, CLI reference, and statistics:

In [ ]:
#| export
def add_project_structure_section(index_path: Path = None,      # Path to index.ipynb
                                 marker: str = "## Project Structure",  # Section marker
                                 exclude_index: bool = True     # Exclude index.ipynb from tree
                                 ) -> str:                       # Generated structure content
    "Generate project structure tree content for index.ipynb"
    if index_path is None:
        cfg = get_config()
        path = cfg.nbs_path
    else:
        path = index_path.parent
    
    # Generate tree with descriptions
    tree_content = generate_tree_with_descriptions(path, exclude_index=exclude_index)
    
    # Get summary statistics
    summary = get_tree_summary(path)
    
    # Create the complete section
    content = f"{marker}\n\n"
    content += "```\n"
    content += tree_content
    content += "\n```\n\n"
    content += summary
    
    return content

In [ ]:
#| export
def add_dependencies_section(index_path: Path = None,           # Path to index.ipynb
                           marker: str = "## Module Dependencies", # Section marker
                           direction: str = "LR"                # Diagram direction
                           ) -> str:                            # Generated dependencies content
    "Generate module dependencies diagram content for index.ipynb"
    if index_path is None:
        cfg = get_config()
        path = cfg.nbs_path
    else:
        path = index_path.parent
    
    # Build dependency graph
    graph = build_dependency_graph(path)
    
    # Generate Mermaid diagram
    diagram = generate_mermaid_diagram(graph, direction=direction)
    
    # Create the complete section
    content = f"{marker}\n\n"
    content += diagram  # diagram already includes ```mermaid wrapper
    content += "\n\n"
    
    # Count unique module pairs (cross-notebook dependencies)
    if graph.dependencies:
        # Get unique source-target pairs
        unique_deps = set()
        for dep in graph.dependencies:
            unique_deps.add((dep.source, dep.target))
        
        content += f"*{len(unique_deps)} cross-module dependencies detected*"
    else:
        content += "No cross-module dependencies detected."
    
    return content

In [ ]:
#| export
import subprocess
import importlib.util
import argparse
from typing import Optional

def add_cli_reference_section(marker: str = "## CLI Reference"  # Section marker
                            ) -> str:                           # Generated CLI content
    "Generate CLI reference content for index.ipynb based on project's console scripts"
    
    cfg = get_config()
    
    # Check if project has console scripts
    console_scripts = getattr(cfg, 'console_scripts', None)
    if not console_scripts:
        return f"{marker}\n\nNo CLI commands found in this project."
    
    content = f"{marker}\n\n"
    
    # Parse console scripts to find CLI commands
    cli_commands = []
    if console_scripts:
        for script in console_scripts.split():
            if '=' in script:
                script_name, module_path = script.split('=', 1)
                cli_commands.append((script_name, module_path))
    
    if not cli_commands:
        return f"{marker}\n\nNo CLI commands found in this project."
    
    # Generate documentation for each CLI command
    for script_name, module_path in cli_commands:
        content += f"### `{script_name}` Command\n\n"
        
        try:
            # Try to get help text by running the command
            result = subprocess.run([script_name, '--help'], 
                                  capture_output=True, text=True, timeout=10)
            if result.returncode == 0:
                # Parse the help output to extract commands
                help_text = result.stdout
                content += "```\n"
                content += help_text
                content += "\n```\n\n"
                
                # Extract subcommands if they exist
                if 'Available commands:' in help_text or 'subcommands:' in help_text.lower():
                    content += f"#### Usage Examples\n\n"
                    content += "```bash\n"
                    
                    # Extract command names from help text
                    lines = help_text.split('\n')
                    in_commands_section = False
                    
                    for line in lines:
                        if 'available commands' in line.lower() or 'subcommands' in line.lower():
                            in_commands_section = True
                            continue
                        elif in_commands_section and line.strip():
                            # Look for command names (typically the first word after whitespace)
                            parts = line.strip().split()
                            if parts and not parts[0].startswith('-'):
                                cmd = parts[0]
                                if cmd not in ['usage:', 'positional', 'optional', 'options:']:
                                    content += f"# {parts[1:] if len(parts) > 1 else ['Command description']}\n"
                                    content += f"{script_name} {cmd}\n\n"
                        elif in_commands_section and not line.strip():
                            break
                    
                    content += "```\n\n"
            else:
                content += f"CLI command `{script_name}` found but help text unavailable.\n\n"
                
        except (subprocess.TimeoutExpired, subprocess.CalledProcessError, FileNotFoundError):
            content += f"CLI command `{script_name}` found but help text unavailable.\n\n"
    
    content += f"For detailed help on any command, use `{cli_commands[0][0]} <command> --help`."
    
    return content

In [ ]:
#| export
def update_index_comprehensive(index_path: Path = None,         # Path to index.ipynb
                              include_structure: bool = True,  # Include project structure
                              include_dependencies: bool = True, # Include module dependencies
                              include_cli: bool = True,         # Include CLI reference
                              include_modules: bool = True      # Include module documentation
                              ) -> None:                        # Updates index.ipynb in place
    "Comprehensively update index.ipynb with project structure, dependencies, CLI, and modules"
    if index_path is None:
        cfg = get_config()
        index_path = cfg.nbs_path / "index.ipynb"
    
    # Read the existing notebook
    nb = read_nb(index_path)
    
    # Define all section markers we want to manage
    section_markers = []
    if include_structure:
        section_markers.append("## Project Structure")
    if include_dependencies:
        section_markers.append("## Module Dependencies") 
    if include_cli:
        section_markers.append("## CLI Reference")
    if include_modules:
        section_markers.append("## Module Overview")
    
    # Remove all existing managed sections
    cells_to_keep = []
    skip_until_next_section = False
    current_skipping_marker = None
    
    for i, cell in enumerate(nb.cells):
        if cell.cell_type == 'markdown':
            source = cell.source.strip()
            
            # Check if this starts any of our managed sections
            section_found = None
            for marker in section_markers:
                if source.startswith(marker):
                    section_found = marker
                    break
            
            if section_found:
                skip_until_next_section = True
                current_skipping_marker = section_found
                continue
            # Check if we've reached a new top-level section (## but not ### or more)
            elif skip_until_next_section and re.match(r'^##\s+(?!#)', source):
                # Make sure it's not another one of our managed sections
                if not any(source.startswith(marker) for marker in section_markers):
                    skip_until_next_section = False
                    current_skipping_marker = None
        
        # Keep the cell if we're not in a managed section
        if not skip_until_next_section:
            cells_to_keep.append(cell)
    
    # Generate new content sections
    new_sections = []
    
    if include_structure:
        try:
            structure_content = add_project_structure_section(index_path)
            structure_cell = mk_cell(structure_content, cell_type='markdown')
            new_sections.append(structure_cell)
        except Exception as e:
            print(f"Error generating project structure: {e}")
    
    if include_dependencies:
        try:
            deps_content = add_dependencies_section(index_path)
            deps_cell = mk_cell(deps_content, cell_type='markdown')
            new_sections.append(deps_cell)
        except Exception as e:
            print(f"Error generating dependencies: {e}")
    
    if include_cli:
        try:
            cli_content = add_cli_reference_section()
            cli_cell = mk_cell(cli_content, cell_type='markdown')
            new_sections.append(cli_cell)
        except Exception as e:
            print(f"Error generating CLI reference: {e}")
    
    if include_modules:
        try:
            # Generate module overview sections
            notebooks = get_notebook_files(index_path.parent, recursive=True)
            
            # Create the module overview header cell
            header_cell = mk_cell("## Module Overview\n\nDetailed documentation for each module in the project:", 
                                 cell_type='markdown')
            new_sections.append(header_cell)
            
            # Sort notebooks by their numeric prefix
            def sort_key(
                nb_path  # TODO: Add type hint and description
            ): # TODO: Add type hint
                "TODO: Add function description"
                match = re.match(r'^(\d+)', nb_path.stem)
                if match:
                    return (int(match.group(1)), nb_path.stem)
                return (999, nb_path.stem)
            
            sorted_notebooks = sorted(notebooks, key=sort_key)
            
            # Generate overview for each module
            for nb_path in sorted_notebooks:
                if nb_path.stem in ['index', '00_index']:
                    continue
                    
                try:
                    module_info = parse_notebook(nb_path)
                    
                    # Only include if it has exported content
                    has_exports = any([
                        any(f.is_exported for f in module_info.functions),
                        any(c.is_exported for c in module_info.classes),
                        any(v.is_exported for v in module_info.variables)
                    ])
                    
                    if has_exports:
                        overview_md = generate_module_overview(module_info)
                        overview_cell = mk_cell(overview_md, cell_type='markdown')
                        new_sections.append(overview_cell)
                except Exception as e:
                    print(f"Error parsing {nb_path}: {e}")
                    continue
        except Exception as e:
            print(f"Error generating module documentation: {e}")
    
    # Rebuild the notebook with new sections at the end
    nb.cells = cells_to_keep + new_sections
    
    # Write the updated notebook
    write_nb(nb, index_path)

In [ ]:
# Example: Update index.ipynb with comprehensive documentation
# update_index_comprehensive()  # Full update with all sections

# Or customize what sections to include:
# update_index_comprehensive(
#     include_structure=True,     # Project tree
#     include_dependencies=True,  # Module dependencies 
#     include_cli=True,          # CLI reference
#     include_modules=True       # Module documentation
# )

## Testing

Let's test the API documentation generation on our project:

In [ ]:
# First, let's export our modules
import nbdev
nbdev.nbdev_export()

In [ ]:
# Test the updated generate_module_overview function
import nbdev
nbdev.nbdev_export()

# Test generating documentation for a single module
core_module = parse_notebook(Path("00_core.ipynb"))
print("Updated module overview with import statements:")
print("=" * 50)
print(generate_module_overview(core_module))

Updated module overview with import statements:
### Core Utilities (`00_core.ipynb`)
> Core utilities and data models for nbdev project overview generation

#### Import

```python
# Import statements not available
```

#### Functions

```python
def get_notebook_files(path: Path = None,           # Directory to search (defaults to nbs_path)
                      recursive: bool = True        # Search subdirectories
                      ) -> List[Path]:              # List of notebook paths
    "Get all notebook files in a directory"
```

```python
def get_subdirectories(path: Path = None,           # Directory to search (defaults to nbs_path)
                      recursive: bool = False       # Include all nested subdirectories
                      ) -> List[Path]:              # List of directory paths
    "Get subdirectories in a directory"
```

```python
def read_notebook(path: Path                    # Path to notebook file
                 ) -> Dict[str, Any]:           # Notebo

In [ ]:
# Test another module to see how the imports look
parsers_module = parse_notebook(Path("01_parsers.ipynb"))
print("Parsers module overview:")
print("=" * 30)
print(generate_module_overview(parsers_module)[:800] + "..." if len(generate_module_overview(parsers_module)) > 800 else generate_module_overview(parsers_module))

Parsers module overview:
### Notebook and Module Parsing (`01_parsers.ipynb`)
> Parse notebook metadata, content, and extract function/class signatures with docments

#### Import

```python
# Import statements not available
```

#### Functions

```python
def extract_docments_signature(node: ast.FunctionDef,          # AST function node
                              source_lines: List[str]          # Source code lines
                              ) -> str:                        # Function signature
    "Extract function signature with docments-style comments"
```

```python
def _parse_decorators(node: Union[ast.ClassDef, ast.FunctionDef]  # AST node with decorators
                     ) -> List[str]:                              # List of decorator names
    "Parse decorators from an AST node"
```

```python
def...


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()